<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center">RFM Customer Analysis</h1>
        <div style="width: 90%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Anjana Ranjan</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="Enter Date" pubdate>July, 2018</time>
        </div>
    </div>
</div>

# Functionality
RFM stands for Recency, Frequency, and Monetary. It is a customer segmentation technique that uses past purchase behavior to divide customers into groups.

RFM Score Calculations
* RECENCY (R): Days since last purchase
* FREQUENCY (F): Total number of purchases
* MONETARY VALUE (M): Total money this customer spent


A score is calculated based on these 3 parameters and the customers are judged based on their respective scores. 
<img src="https://cdn-images-1.medium.com/max/800/1*MeXvFG4Ez0wStMDCXZyN-Q.png" />

Each segment of customers is aptly dealt with, 

For example:

**Best Customers - Champions**: Reward them. They can be early adopters to new products. Suggest them "Refer a friend".

**At Risk**: Send them personalized emails to encourage them to shop.

##  Required parameters
* Input Dataframe which consits of sales data.
* The dataframe should contain the equivalent of the following attributes/columns:
    * Customer ID(Unique ID of each customer)
    * Date of transaction
    * Revenue: consists the cost of all units purchased(Optional: can be provided instead of 'Unit Price') 
    * Quantity: No. of items purchased
    * Unit Price: If the total Revenue is not present in the dataset. the Unit Price attribute should be given. Used      instead of 'Revenue' attribute
    * Invoice Number: The number provided to keep track of the sales transactions

## Input parameters
* DataFrame containing sales data
* Following column names must be entered:
    * customerID=
    * date=
    * revenue=(Optional, used in case no 'unitprice' column present)
    * quantity=
    * unitprice=(Optional, used in case no 'revenue' column present)
    * invoiceNO

## return
* DataFrame containing scores of each customer
* An analysis of number of Best Customers, Loyal Customers,
 Big Spenders,
 Almost Lost,
 Lost Customers, and
 Lost Cheap Customers.

# Code

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

import time, warnings
import datetime as dt

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

In [2]:
retail_df = pd.read_csv('sales.csv',encoding="ISO-8859-1",dtype={'CustomerID': str,'InvoiceID': str})
retail_df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01-12-2010 08:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,01-12-2010 08:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,01-12-2010 08:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01-12-2010 08:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,01-12-2010 08:26,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2.0,01-12-2010 08:26,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6.0,01-12-2010 08:26,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6.0,01-12-2010 08:28,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6.0,01-12-2010 08:28,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32.0,01-12-2010 08:34,1.69,13047,United Kingdom


In [3]:
sales = pd.read_csv('sales_data_sample.csv',encoding="ISO-8859-1")
sales.head(10)

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium
5,10168,36,96.66,1,3479.76,10/28/2003 0:00,Shipped,4,10,2003,...,9408 Furth Circle,NaN,Burlingame,CA,94217,USA,NaN,Hirano,Juri,Medium
6,10180,29,86.13,9,2497.77,11/11/2003 0:00,Shipped,4,11,2003,...,"184, chausse de Tournai",NaN,Lille,NaN,59000,France,EMEA,Rance,Martine,Small
7,10188,48,100.00,1,5512.32,11/18/2003 0:00,Shipped,4,11,2003,...,"Drammen 121, PR 744 Sentrum",NaN,Bergen,NaN,N 5804,Norway,EMEA,Oeztan,Veysel,Medium
8,10201,22,98.57,2,2168.54,12/1/2003 0:00,Shipped,4,12,2003,...,5557 North Pendale Street,NaN,San Francisco,CA,NaN,USA,NaN,Murphy,Julie,Small
9,10211,41,100.00,14,4708.44,1/15/2004 0:00,Shipped,1,1,2004,...,"25, rue Lauriston",NaN,Paris,NaN,75016,France,EMEA,Perrier,Dominique,Medium


In [4]:
df1 = pd.read_csv('Superstore.csv',encoding="ISO-8859-1")
df1.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08-11-2016,11-11-2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,08-11-2016,11-11-2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,12-06-2016,16-06-2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,11-10-2015,18-10-2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,11-10-2015,18-10-2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [5]:
class rfm:
    
    def __init__(self, df=None, customerID=None, date=None, revenue= None, quantity=None, unitprice=None, invoiceNO=None):
        self.df= df
        #self.revenue= revenue
        self.quantity= quantity
        self.customerID= customerID
        self.date = date
        self.unitprice= unitprice
        self.invoiceNO= invoiceNO
        self.revenue= revenue

    def preprocess(self):
        retail_uk = self.df
        retail_uk = retail_uk[retail_uk[self.quantity]>0]
        retail_uk.dropna(subset=[self.customerID],how='all',inplace=True)
        return retail_uk

    def recency(self):
        retail_uk = self.preprocess()
        retail_uk['date'] = pd.DatetimeIndex(retail_uk[self.date]).date
        now = retail_uk['date'].max()
        recency_df = retail_uk.groupby(by=self.customerID, as_index=False)['date'].max()
        recency_df.columns = ['CustomerID','LastPurshaceDate']
        recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)
        recency_df.drop('LastPurshaceDate',axis=1,inplace=True)
        return recency_df

    def frequency(self):
        retail_uk= self.preprocess()
        retail_uk_copy = retail_uk
        retail_uk_copy.drop_duplicates(subset=[self.invoiceNO, self.customerID], keep="first", inplace=True)
        #calculate frequency of purchases
        frequency_df = retail_uk_copy.groupby(by=[self.customerID], as_index=False)[self.invoiceNO].count()
        frequency_df.columns = ['CustomerID','Frequency']
        return frequency_df
    
    def monetaryvalue(self):
        retail_uk= self.preprocess()
        if self.revenue != None:
            retail_uk['TotalCost'] = retail_uk[self.revenue]
            monetary_df = retail_uk.groupby(by=self.customerID,as_index=False).agg({'TotalCost': 'sum'})
            monetary_df.columns = ['CustomerID','Monetary Value']
            return monetary_df
        else:
            retail_uk['TotalCost'] = retail_uk[self.quantity] * retail_uk[self.unitprice]
            monetary_df = retail_uk.groupby(by=self.customerID,as_index=False).agg({'TotalCost': 'sum'})
            monetary_df.columns = ['CustomerID','Monetary Value']
            return monetary_df

    def rfmtable(self):
        a = self.recency()
        b = self.frequency()
        c = self.monetaryvalue()
        temp_df = a.merge(b,on='CustomerID')
        rfm_df = temp_df.merge(c,on='CustomerID')
        return rfm_df
    
    def rfmsegmentation(self):
        rfm_df= self.rfmtable()
        rfm_segmentation = rfm_df
        quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
        quantiles.to_dict()
        def RScore(x,p,d):
            if x <= d[p][0.25]:
                return 4
            elif x <= d[p][0.50]:
                return 3
            elif x <= d[p][0.75]: 
                return 2
            else:
                return 1
        # Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
        def FMScore(x,p,d):
            if x <= d[p][0.25]:
                return 1
            elif x <= d[p][0.50]:
                return 2
            elif x <= d[p][0.75]: 
                return 3
            else:
                return 4
        rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
        rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
        rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary Value'].apply(FMScore, args=('Monetary Value',quantiles,))
        rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
        display(rfm_segmentation)
        print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
        print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
        print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
        print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
        print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
        print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))


# Testing 


## Test 1

In [6]:
r = rfm(df= retail_df, quantity= 'Quantity', customerID='CustomerID', date= 'InvoiceDate', unitprice= 'UnitPrice', invoiceNO='InvoiceNo')
r.recency()

,CustomerID,Recency
0,12346,324
1,12347,93
2,12348,218
3,12350,309
4,12352,261
5,12353,203
6,12354,231
7,12355,94
8,12356,126
9,12358,1


In [7]:

r.frequency()

,CustomerID,Frequency
0,12346,1
1,12347,5
2,12348,3
3,12350,1
4,12352,5
5,12353,1
6,12354,1
7,12355,1
8,12356,2
9,12358,1


In [8]:

r.monetaryvalue()

,CustomerID,Monetary Value
0,12346,77183.60
1,12347,2790.86
2,12348,1487.24
3,12350,334.40
4,12352,1561.81
5,12353,89.00
6,12354,1079.40
7,12355,459.40
8,12356,2753.08
9,12358,484.86


In [9]:
r.rfmtable()

,CustomerID,Recency,Frequency,Monetary Value
0,12346,324,1,77183.60
1,12347,93,5,2790.86
2,12348,218,3,1487.24
3,12350,309,1,334.40
4,12352,261,5,1561.81
5,12353,203,1,89.00
6,12354,231,1,1079.40
7,12355,94,1,459.40
8,12356,126,2,2753.08
9,12358,1,1,484.86


In [10]:
r.rfmsegmentation()

,CustomerID,Recency,Frequency,Monetary Value,R_Quartile,F_Quartile,M_Quartile,RFMScore
0,12346,324,1,77183.60,1,1,4,114
1,12347,93,5,2790.86,4,4,4,444
2,12348,218,3,1487.24,2,3,4,234
3,12350,309,1,334.40,1,1,2,112
4,12352,261,5,1561.81,1,4,4,144
5,12353,203,1,89.00,2,1,1,211
6,12354,231,1,1079.40,2,1,3,213
7,12355,94,1,459.40,4,1,2,412
8,12356,126,2,2753.08,3,2,4,324
9,12358,1,1,484.86,4,1,2,412


Best Customers:  303
Loyal Customers:  651
Big Spenders:  830
Almost Lost:  47
Lost Customers:  4
Lost Cheap Customers:  362


### Pass/Fail
Test Passed

## Test 2

In [11]:
s = rfm(df= sales, quantity= 'QUANTITYORDERED', customerID='CONTACTLASTNAME', date= 'ORDERDATE', unitprice= 'PRICEEACH', invoiceNO='ORDERNUMBER')
s.rfmsegmentation()

,CustomerID,Recency,Frequency,Monetary Value,R_Quartile,F_Quartile,M_Quartile,RFMScore
0,Accorti,264,2,72645.01,1,1,2,112
1,Ashworth,195,3,136873.25,2,2,4,224
2,Barajas,263,2,48385.01,1,1,1,111
3,Benitez,89,3,77027.44,3,2,2,322
4,Bennett,211,2,73170.13,2,1,2,212
5,Berglund,193,4,58374.21,2,3,2,232
6,Bergulfsen,207,4,95223.90,2,3,3,233
7,Bertrand,192,3,73965.52,2,2,2,222
8,Brown,58,10,272790.97,4,4,4,444
9,Calaghan,118,3,46399.18,3,2,1,321


Best Customers:  6
Loyal Customers:  14
Big Spenders:  19
Almost Lost:  3
Lost Customers:  0
Lost Cheap Customers:  12


### Pass

## Test 3

In [12]:
v = rfm(df= df1, quantity= 'Quantity', customerID='Customer ID', date= 'Order Date', unitprice= 'Sales', invoiceNO='Order ID')
v.rfmsegmentation()

,CustomerID,Recency,Frequency,Monetary Value,R_Quartile,F_Quartile,M_Quartile,RFMScore
0,AA-10315,184,5,16073.3440,2,1,3,213
1,AA-10375,48,9,2734.8300,3,4,1,341
2,AA-10480,259,4,5493.9360,1,1,2,112
3,AA-10645,233,6,22927.8550,1,2,4,124
4,AB-10015,445,3,1984.4800,1,1,1,111
5,AB-10060,105,8,35390.5800,2,3,4,234
6,AB-10105,41,10,143857.7090,3,4,4,344
7,AB-10150,41,5,3111.2420,3,1,1,311
8,AB-10165,106,8,3625.5140,2,3,1,231
9,AB-10255,166,9,2397.5640,2,4,1,241


Best Customers:  29
Loyal Customers:  159
Big Spenders:  198
Almost Lost:  16
Lost Customers:  7
Lost Cheap Customers:  64


### Pass